In [101]:
#Autores: Daniel Castillo, Karla Salas
from os import listdir
from os.path import isfile, join
#Para ver las palabras
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
# nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split #particiones
from nltk.corpus import stopwords #Listas de stopwords
from nltk.tokenize import word_tokenize #Tokens
import re #regex
from itertools import chain #bigramas

[nltk_data] Downloading package punkt to /home/karla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/karla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Limpiamos y tokenizamos
(Tomamos el código de la práctica anterior)

Tokenizamos por palabra, ya que nos interesa medir la probabilidad de transitar de una palabra a otra y no por subpalabras, para intentar, más adelante, responder con una oración dada una entrada.

In [96]:
def get_txt(path):
    """
    Regresa una lista con el contenido de todos los archivos de un directorio

    Args:
        path (str): ruta de la carpeta
    """
    text = []
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    
    for file in onlyfiles:
         with open(path+"/"+file, 'rb') as f:
            text.append(f.read().decode('utf-8', 'replace'))
    return text

# Guardamos cada película en un diccionario
# cada entrada del diccionario es una lista con las peliculas leídas
movies = {}
movies["Pride & Prejudice"] = get_txt("../corpus/Pride & Prejudice")
movies["Marvel"] = get_txt("../corpus/Marvel")
movies["Christopher Nolan"] = get_txt("../corpus/Christopher Nolan")

In [97]:
# tokens que necesitan ser limpiados del corpus y que no
# se encuentran en la lista de stopwords
more = ["_","-","'ve", "'ll", "'t", "'s", "'re", "'", "'m", "'d", "n't", "oh", "hey", "yeah","okay", "mr.", "miss", "mrs."]
stopwords_list = stopwords.words('english') + more

def get_tokens_clean(text):
    '''
    Genera los tokens de una cadena y los limpia
    (quita símbolos raros y stopwords)
    
    Args:
        text (str): cadena
    '''
    tokens = word_tokenize(text)
    clean = []
    pattern = r'[^a-z0-9\s]'
    for w in tokens:
        #quita stopwords y convierte a minúsculas
        w = re.sub(pattern,'', w.lower())
        if w not in stopwords_list and w != '':
            if  w == "na": #Para juntar gon na, wan na, etc.
                clean[-1] += w
            else:
                clean.append(w)

    return clean

In [98]:
def merge_movies(col):
    '''
    Mezcla el corpus de cada película en una entrada del
    diccionario movies en un solo corpus, 
    guarda los diálogos por línea tokenizados y limpios

    Args:
        col (dic): Diccionario con los diálogos
    '''
    corpus = []
    #Iteramos sobre las películas
    for movie in col:
        print(movie, len(movies[movie]))
        for text in movies[movie]:
            #Cada texto se guarda por línea (contexto)
            #Quitamos líneas vacías
            corpus += map(get_tokens_clean, list(filter(bool, text.split('\n'))))
    return corpus

all_movies = merge_movies(movies)

Pride & Prejudice 1
Marvel 23
Christopher Nolan 5


In [100]:
#Particiones train y test (30%)
train, test = train_test_split(all_movies, test_size=0.3)
print('Número de cadenas train:',len(train))
print('Número de cadenas test:',len(test))
#Falta hacer algo con las palabras fuera del vocabulario

Número de cadenas train: 39846
Número de cadenas test: 17078


## Vocabulario (indexado)
Obtenemos los pares de entrenamiento a partir de contextos, bigramas.

In [ ]:
def get_vocabulary():

voc = vocab()
#Creamos el vocabulario y le asignamos un indice a cada simbolo segun su aparicion
cads_idx = list(text2numba(corpus_hapax,voc))
#Indicamos las etiquetas a usar
EOS = '<EOS>'
BOS = '<BOS>'
#Cada etiqeuta se le asigna un indice numerico
BOS_IDX = max(voc.values())+2
EOS_IDX = max(voc.values())+1
#Se agregan estas etiqeutas al vocabulario
voc[EOS] = EOS_IDX
voc[BOS] = BOS_IDX
#A cada línea de los diálogos se le agrega la etiqueta BOS al inicio y EOS al final
cadenas = [[BOS_IDX] + text + [EOS_IDX] for text in train]
#Se crean los bigramas
bigrams = list(chain(*[zip(cad,cad[1:]) for cad in cadenas]))

print(bigrams[:10])
